In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
import time
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas

In [2]:
from google.colab import drive

drive.mount('/content/drive') 

Mounted at /content/drive


In [3]:
%cd '/content/drive/My Drive/FSI_Problem'
!ls

/content/drive/My Drive/FSI_Problem
Bert_Sentiment_AmazonDataset.ipynb	 PhoBERT_base_transformers.tar.gz
Clone_Foody.ipynb			 PhoBert_Sentiment_Foody.ipynb
Concat_Data.ipynb			 Test_Foody.ipynb
Data					 train_dataloader.pkl
MultiBert_Sentiment_AmazonDataset.ipynb  val_dataloader.pkl
PhoBERT_base_fairseq			 vncorenlp
PhoBERT_base_fairseq.tar.gz		 X1.pkl
PhoBERT_base_transformers		 y1.pkl


In [4]:
def crawl_comment(url):
  html = urlopen(url)
  soup = BeautifulSoup(html, 'html')
  comments = [] 
  title = []
  marks = []
  comments = [i.find_all('span')[0].text for i in soup.find_all(class_ = 'rd-des')][:len(comments)-2]
  title = [i.text for i in soup.find_all(class_ = 'rd-title')][:len(title)-2]
  marks = [i.find_all('span')[0].text for i in soup.find_all(class_= "review-points")][:len(marks)-3]
  
  return comments,title,marks 

In [5]:
def convert_link(name):
  return name.replace(' ', '+') 
def convert_province(province):
  return province.replace(' ','-')

In [6]:
def get_link_dish(province, dish_name,page):
  url = "https://www.foody.vn/"+convert_province(province)+"/food/dia-diem?q=" +convert_link(dish_name) +"&ss=header_search_form"+"&page="+str(page)
  html = urlopen(url)
  soup = BeautifulSoup(html, 'html')
  
  links = []
  for i in soup.find_all('a', href=True):
    if 'target="_blank"' in str(i):
      links.append(i['href'])

  index_star = []
  for index, link in enumerate(links):
    if link == '#':
      index_star.append(index)

  return links[index_star[0]+1:index_star[len(index_star)-1]:2]


In [7]:
def clone_link_city(city,dish_name):
  all_link = []
  for i1 in city:
    total_link_city = []
    for i2 in range(1,10,1):
      total_link_city.append(get_link_dish(i1,dish_name,i2))
    total_link_city_flat = [y for x in total_link_city for y in x]
    all_link.append(total_link_city_flat)
  all_link_flat = [y for x in all_link for y in x] 

  all_link_unique = list(dict.fromkeys(all_link_flat))
  return all_link_unique

  

In [8]:
def clone_comment(all_link_unique):
  total_comments = []
  total_title = []
  total_marks = []
  comments = []
  title = []
  marks = []
  for index, i in enumerate(all_link_unique): 
    url_binh_luan = 'https://www.foody.vn/'+i+'/binh-luan' 
    comments,title,marks = crawl_comment(url_binh_luan)
    if len(comments) != len(marks):
      continue
    total_comments.append(comments)
    total_title.append(title)
    total_marks.append(marks)
    
  total_comment_flat = [y for x in total_comments for y in x]
  total_title_flat = [y for x in total_title for y in x]
  total_marks_flat = [y for x in total_marks for y in x]

  return total_comment_flat, total_title_flat, total_marks_flat
  

In [12]:
city = ['ho chi minh','ha noi','hai phong','da nang','can tho','khanh hoa','vung tau','dong nai','hue','binh duong']
dish_name = 'thi cho'
all_link_unique = clone_link_city(city,dish_name)
total_comment_flat, total_title_flat, total_marks_flat=clone_comment(all_link_unique)
a = pandas.DataFrame({'comment':total_comment_flat, 
                  'title':total_title_flat, 
                  'marks':total_marks_flat
                  }) 
url = '/content/drive/My Drive/FSI_Problem/Data/'+dish_name+'.csv'
a.to_csv(url, header=True, index=False)